## TSNE Analysis of VirProBERT Few Shot Classifier
### Dataset: EMBL mapping, Vertebrates, Non-IDV

**Models**: VirProBERT, Few-Shot Classifier

**Maximum Sequence Length**: 2048

In [1]:
import sys
import os
sys.path.append(os.path.join(os.getcwd(), "..", "..", "..", "..", ".."))
sys.path.append(os.path.join(os.getcwd(), "..", "..", "..", ".."))
sys.path.append(os.path.join(os.getcwd(), "..", "..", ".."))
sys.path.append(os.path.join(os.getcwd(), "..", ".."))
sys.path

['/home/blessyantony/dev/git/zoonosis/src/jupyter_notebooks/few_shot_learning',
 '/opt/conda/lib/python38.zip',
 '/opt/conda/lib/python3.8',
 '/opt/conda/lib/python3.8/lib-dynload',
 '',
 '/home/blessyantony/.local/lib/python3.8/site-packages',
 '/opt/conda/lib/python3.8/site-packages',
 '/opt/conda/lib/python3.8/site-packages/IPython/extensions',
 '/home/blessyantony/.ipython',
 '/home/blessyantony/dev/git/zoonosis/src/jupyter_notebooks/few_shot_learning/../../../../..',
 '/home/blessyantony/dev/git/zoonosis/src/jupyter_notebooks/few_shot_learning/../../../..',
 '/home/blessyantony/dev/git/zoonosis/src/jupyter_notebooks/few_shot_learning/../../..',
 '/home/blessyantony/dev/git/zoonosis/src/jupyter_notebooks/few_shot_learning/../..']

In [2]:
from src.utils import utils, nn_utils

import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.manifold import TSNE
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path

from src.utils import utils, nn_utils

In [3]:
virprobert_model_file_path = os.path.join(os.getcwd(), "..", "..", "..", "..", "..", "output/raw/uniref90_embl_vertebrates_non_idv_t0.01_c5/20240611/host_multi/fine_tuning/host_prediction_fnn_2l_d1024_lr1e-4_itr0.pth")
virprobert_few_shot_classifier_model_file_path = os.path.join(os.getcwd(), "..", "..", "..", "..", "..", "output/raw/uniref90_embl_vertebrates_non_idv/20240612/host_multi/few_shot_learning/virprobert_itr2.pth")

In [ ]:
virprobert_settings = {
    mlm_encoder_settings: {
        
    },
    host_prediction_settings: {
        
    }
}

